### Two Agents Vault

In this problem there are two agents, A and B, that have to grab an object inside a vault.

The vault is closed, and A has the key to open it. On the other hand, A cannot enter the vault, but B can.

The idea is to ask each agent for a PDDL problem + domain of their partial plans to solve the problem.

The problems are integrated by the Orchestrator and then fed to a solver (Fast Downwards).

We eventually find the optimal solution to the problem in a fully automated way.

In [37]:
from src.llm_plan.LLM import GPT_Ollama
from src.llm_plan.Parser import PDDLParser
from src.llm_plan.Problem import ProblemStaticAgentsVault
from src.llm_plan.StaticEnvironment import StaticAgentsVault

model = GPT_Ollama()
env = StaticAgentsVault(grid_size=4, visibility=0)
problem = ProblemStaticAgentsVault(env)
parser = PDDLParser()

env.render()

--- Static Agents Vault Environment ---
Goal: {'Agent A': 'Open the vault and grab the object inside.', 'Agent B': 'Open the vault and grab the object inside.'}

--- Positions ---
Agent A: (3, 3)
Agent B: (3, 1)
Vault: (3, 2)
Agent A sees B: False
Agent A sees B: False

--- Knowledge of Agents ---
Agent A:
  - I am a big robot.
  - The vault is closed.
  - I have the key to open the vault.
  - I cannot grab the object inside the vault.
Agent B:
  - I am a small robot
  - The vault is closed.
  - I do not have the key to open the vault.
  - If the vault is open, I can enter and grab the object.

--- Public Information ---
 - There is a vault in the environment.
 - The vault is closed.
 - The vault requires a key to be opened.
 - The entrance of the vault is small.

--- Grid ---
. . . .
. . . B
. . . V
. . . A
--- Legend ---
--- A: Agent A, B: Agent B, V: Vault ---
-------------------------------------


In [38]:
# Generate the PDDL plans for each agent
agents = ["Agent A", "Agent B"]
agent_prompts = {agent: "" for agent in agents}
agent_responses = {agent: "" for agent in agents}

for agent in agents:
    print(f"Prompting {model.model_name} with the following prompt:\n")
    print(problem.prompts[agent])
    agent_prompts[agent] = f"{agent} Problem:\n" + problem.prompts[agent] + "\n"
    answer = model.generate_sync(problem.system_prompts[agent], 
                                 problem.prompts[agent])
    print("Answer:")
    print(answer)
    agent_responses[agent] = f"{agent} Response:\n" + answer + "\n"
    print("-"*30 + "\n")

Prompting gpt-oss-20B-ollama with the following prompt:

You are Agent A. You are in an enviroment with the following public information:
There is a vault in the environment.
The vault is closed.
The vault requires a key to be opened.
The entrance of the vault is small.

You have the following knowledge:
I am a big robot.
The vault is closed.
I have the key to open the vault.
I cannot grab the object inside the vault.

This is the global goal to solve:Open the vault and grab the object inside.

Think step by step and and provide a PDDL domain and a PDDL problem file to solve the task.
If you miss some information, do not make assumptions,just give a plan that concerns the information you have.
Error while generating a response: Connection error.
Answer:
Connection error.
------------------------------

Prompting gpt-oss-20B-ollama with the following prompt:

You are Agent B. You are in an enviroment with the following public information:
There is a vault in the environment.
The vault i

In [ ]:
# Prompt the orchestrator for the final plan
orchestrator_prompt = problem.prompts["Orchestrator"].format(pddl_agent_A=agent_prompts["Agent A"] + agent_responses["Agent A"],
                                                             pddl_agent_B=agent_prompts["Agent B"] + agent_responses["Agent B"],
                                                             goal=env.goal["Agent A"])

final_plan = model.generate_sync(problem.system_prompts["Orchestrator"], 
                                 orchestrator_prompt)

print(final_plan)                           

In [ ]:
# Save the final plan to a file
with open("./tmp/final_plan_in_text.txt", "w") as f:
    f.write(final_plan)

# Isolate the pddl problem and domain
pddl_domain, pddl_problem = parser.parse("./tmp/final_plan_in_text.txt")

In [ ]:
# Save the PDDL domain and problem to files
with open("./tmp/domain.pddl", "w") as domain_file:
    domain_file.write(pddl_domain)

with open("./tmp/problem.pddl", "w") as problem_file:
    problem_file.write(pddl_problem)


In [36]:
import subprocess

# Invoke fast downward and solve the problem
command = "./solvers/fast-downward-24.06.1/fast-downward.py --alias lama-first --plan-file ./tmp/sas_plan ./tmp/domain.pddl ./tmp/problem.pddl > ./tmp/logs.txt 2>&1"

subprocess.run(command, shell=True)

CompletedProcess(args='./solvers/fast-downward-24.06.1/fast-downward.py --alias lama-first --plan-file ./tmp/sas_plan ./tmp/domain.pddl ./tmp/problem.pddl > ./tmp/logs.txt 2>&1', returncode=0)